# NN back propagation（神经网络反向传播）

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import matplotlib
import scipy.optimize as opt
from sklearn.metrics import classification_report#这个包是评价报告
from sklearn.preprocessing import OneHotEncoder

## 数据读取及处理

In [3]:
def load_data(path, transpose=True):
    '''
    数据读取函数
    '''
    data = sio.loadmat(path)
    y = data.get('y')  # (5000,1)
    y = y.reshape(y.shape[0])  # make it back to column vector

    X = data.get('X')  # (5000,400)

    if transpose:
        # for this dataset, you need a transpose to get the orientation right
        X = np.array([im.reshape((20, 20)).T for im in X])

        # and I flat the image again to preserve the vector presentation
        X = np.array([im.reshape(400) for im in X])

    return X, y

In [4]:
X_original, y_original = load_data('ex4data1.mat', transpose=False)
print(X_original.shape,y_original.shape)

(5000, 400) (5000,)


In [5]:
X = np.insert(X_original, 0, np.ones(X_original.shape[0]), axis=1)#增加全部为1的一列
X.shape

(5000, 401)

In [6]:
encoder = OneHotEncoder(sparse=False)
Y = encoder.fit_transform(y_original.reshape(-1,1))
print(Y.shape, Y[0])

(5000, 10) [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [7]:
def load_weight(path):
    data = sio.loadmat(path)
    return data['Theta1'], data['Theta2']

In [8]:
theta_1, theta_2 = load_weight('ex4weights.mat')
theta_1.shape, theta_2.shape

((25, 401), (10, 26))

In [9]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [10]:
# 将2个theta扁平化
def serialize(a, b):
    return np.concatenate((np.ravel(a), np.ravel(b)))

In [11]:
theta = serialize(theta_1, theta_2)  # 扁平化参数，25*401+10*26=10285
theta.shape

(10285,)

In [12]:
def deserialize(theta):
    theta_1 = theta[:25*401].reshape(25,401)
    theta_2 = theta[25*401:].reshape(10, 26)
    return theta_1,theta_2

In [13]:
def feed_forward(theta,X):
    theta_1,theta_2 = deserialize(theta)
    # z表示该层的输入，a表示该层的输出
    a1 = X
    # 第二层
    z2 = X @ theta_1.T
    a2 = np.insert(sigmoid(z2),0,np.ones(len(X)),axis=1)
    #第三次 output层
    z3 = a2 @ theta_2.T
    output = sigmoid(z3)    
    return a1,z2,a2,z3,output

In [14]:
_, _, _, _, output = feed_forward(theta, X)
output

array([[1.12661530e-04, 1.74127856e-03, 2.52696959e-03, ...,
        4.01468105e-04, 6.48072305e-03, 9.95734012e-01],
       [4.79026796e-04, 2.41495958e-03, 3.44755685e-03, ...,
        2.39107046e-03, 1.97025086e-03, 9.95696931e-01],
       [8.85702310e-05, 3.24266731e-03, 2.55419797e-02, ...,
        6.22892325e-02, 5.49803551e-03, 9.28008397e-01],
       ...,
       [5.17641791e-02, 3.81715020e-03, 2.96297510e-02, ...,
        2.15667361e-03, 6.49826950e-01, 2.42384687e-05],
       [8.30631310e-04, 6.22003774e-04, 3.14518512e-04, ...,
        1.19366192e-02, 9.71410499e-01, 2.06173648e-04],
       [4.81465717e-05, 4.58821829e-04, 2.15146201e-05, ...,
        5.73434571e-03, 6.96288990e-01, 8.18576980e-02]])

In [15]:
def cost(theta, X, y):
    # get the data size m
    _, _, _, _, h = feed_forward(theta, X)
    pair_computation = -np.multiply(y, np.log(h)) - np.multiply((1 - y), np.log(1 - h))
    return pair_computation.sum() / len(X)

In [16]:
cost(theta, X, Y)

0.2876291651613189

In [19]:
def cost_regularize(theta, X, y,learn_rate=1):
    theta_1, theta_2 = deserialize(theta)  # t1: (25,401) t2: (10,26)
    m = X.shape[0]
    reg_t1 = (learn_rate / (2 * m)) * np.power(theta_1[:, 1:], 2).sum()  # this is how you ignore first col
    reg_t2 = (learn_rate / (2 * m)) * np.power(theta_2[:, 1:], 2).sum()
    return cost(theta, X, y) + reg_t1 + reg_t2

In [20]:
cost_regularize(theta, X, Y)

0.38376985909092365

In [27]:
def sigmoid_gradient(z):
    """
    pairwise op is key for this to work on both vector and matrix
    """
    return np.multiply(sigmoid(z), 1 - sigmoid(z))

In [43]:
def gradient(theta, X, y):
    # 就是计算每一次两组theta应该减少多少
    theta_1,theta_2 = deserialize(theta) #(25, 401), (10, 26)
    m = len(X)
    # 这就是最终返回的应该减去多少的值
    delta_1 = np.zeros_like(theta_1)
    delta_2 = np.zeros_like(theta_2)
    
    a_1, z_2, a_2, z_3, predicted = feed_forward(theta, X)
    # 一个样本一个样本的累加
    for i in range(m):
        a_1_i = a_1[i, :]  # 输入层 (1, 401) 
        z_2_i = z_2[i, :]  # 第二层的输入(1, 25)
        a_2_i = a_2[i, :]  # 第二层的输出(1, 26)
        predicted_i = predicted[i, :]    # 预测层(1, 10)
        y_i = y[i, :]    # 样本真实值(1, 10)
        # 求预测层和中间隐藏层的预测差值
        diff_3_i = predicted_i - y_i # (1, 10)
        z_2_i = np.insert(z_2_i, 0, np.ones(1))
        diff_2_i = np.multiply(theta_2.T @ diff_3_i,sigmoid_gradient(z_2_i))
        
        delta_2 += np.matrix(diff_3_i).T @ np.matrix(a_2_i)  # (1, 10).T @ (1, 26) -> (10, 26)
        delta_1 += np.matrix(diff_2_i[1:]).T @ np.matrix(a_1_i)  # (1, 25).T @ (1, 401) -> (25, 401)

    delta_1 = delta_1 / m
    delta_2 = delta_2 / m

    return serialize(delta_1, delta_2)

In [44]:
d1, d2 = deserialize(gradient(theta, X, Y))

In [57]:
def expend_array(x):
    return np.array(np.matrix(np.ones(x.shape[0])).T @ np.matrix(x))

# 梯度校验
<img style="float: left;" src="../img/gradient_checking.png">

In [59]:
def check_gradient(theta, X,y,epsilon, regularized=False):
    def a_numeric_grad(plus, minus, regularized=False):
        """calculate a partial gradient with respect to 1 theta"""
        if regularized:
            return (cost_regularize(plus, X, y) - cost_regularize(minus, X, y)) / (epsilon * 2)
        else:
            return (cost(plus, X, y) - cost(minus, X, y)) / (epsilon * 2)
    theta_matrix = expand_array(theta)  # expand to (10285, 10285)
    epsilon_matrix = np.identity(len(theta)) * epsilon
    plus_matrix = theta_matrix + epsilon_matrix
    minus_matrix = theta_matrix - epsilon_matrix
    # 计算约等号两边的值
    numeric_grad = np.array([a_numeric_grad(plus_matrix[i], minus_matrix[i], regularized)
                                    for i in range(len(theta))])

    # analytical grad will depend on if you want it to be regularized or not
    analytic_grad = regularized_gradient(theta, X, y) if regularized else gradient(theta, X, y)
    diff = np.linalg.norm(numeric_grad - analytic_grad) / np.linalg.norm(numeric_grad + analytic_grad)

    print('If your backpropagation implementation is correct,\nthe relative difference will be smaller than 10e-9 (assume epsilon=0.0001).\nRelative Difference: {}\n'.format(diff))

# regularized gradient
Use normal gradient + regularized term

<img style="float: left;" src="../img/nn_reg_grad.png">

In [60]:
def regularized_gradient(theta, X, y, l=1):
    """don't regularize theta of bias terms"""
    m = X.shape[0]
    delta1, delta2 = deserialize(gradient(theta, X, y))
    t1, t2 = deserialize(theta)

    t1[:, 0] = 0
    reg_term_d1 = (l / m) * t1
    delta1 = delta1 + reg_term_d1

    t2[:, 0] = 0
    reg_term_d2 = (l / m) * t2
    delta2 = delta2 + reg_term_d2

    return serialize(delta1, delta2)

In [61]:
def random_init(size):
    return np.random.uniform(-0.12, 0.12, size)

In [70]:
def nn_training(X, y):
    """regularized version
    the architecture is hard coded here... won't generalize
    """
    init_theta = random_init(10285)  # 25*401 + 10*26

    res = opt.minimize(fun=cost_regularize,
                       x0=init_theta,
                       args=(X,Y, 1),
                       method='TNC',
                       jac=regularized_gradient,
                       options={'maxiter': 400})
    return res

In [71]:
res = nn_training(X, Y)#慢
res

     fun: 0.3136198591123115
     jac: array([-8.65416595e-05, -7.55279286e-08, -2.60001165e-07, ...,
       -1.09773604e-05,  4.96631233e-05,  9.88057706e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 400
     nit: 27
  status: 3
 success: False
       x: array([ 0.00000000e+00, -3.77639643e-04, -1.30000583e-03, ...,
       -4.16178230e+00,  1.77502417e+00,  8.33586961e-01])